## Collapse Board Data to School Level, Merge with IPEDS and NACUBO Data

### For Tables 3 and 4 of Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [1]:
use ///
"/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_members_all_long20220501.dta" ///
, clear
keep unitid firstname middlename lastname suffix undergrad mba jd md phd otherprofdegree ///
alumni schoolname schooltype industry_code financetype unitid ///
firmid firmname_new organization forbes forbesyear betweenness strength year eigenvector ///
Edu_sci_med Non_finance Nonprofit Other_fin Alt_fin Public_sector employer_type_aggregated ///
Real_estate industry_jenk alternative

save d_trustees_all_long, replace

file d_trustees_all_long.dta saved


In [1]:
use d_trustees_all_long, clear

In [2]:
describe, fullnames


Contains data from d_trustees_all_long.dta
 Observations:        34,681                  
    Variables:            35                  14 Feb 2025 14:22
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
unitid          double  %9.0g                 unitid
firstname       str16   %19s                  firstname
lastname        str61   %61s                  lastname
year            double  %10.0g                
schoolname      str43   %43s                  schoolname
organization    str353  %353s                 
industry_code   double  %10.0g                
suffix          str3    %9s                   
schooltype      str12   %12s                  
industry_jenk   str14   %14s                  (first) industry
financetype     str18   %-9s                  
middlename      s

In [3]:
** drop non universities and schools not in the top 30
drop if schooltype=="liberal arts"
drop if unitid==232186 | unitid==126818 | unitid==209542 ///
| unitid==221759 | unitid==102614 | unitid==129020 | unitid==181464

keep if year>2002 | year==1989

encode employer_type_aggregated, gen(employer_type_aggregated_num)
codebook employer_type_aggregated_num

(1,886 observations deleted)
(1,689 observations deleted)
(3,897 observations deleted)

-------------------------------------------------------------------------------
employer_type_aggregated_num                                        (unlabeled)
-------------------------------------------------------------------------------

                  Type: Numeric (long)
                 Label: employer_type_aggregated_num

                 Range: [1,8]                         Units: 1
         Unique values: 8                         Missing .: 0/27,209

            Tabulation: Freq.   Numeric  Label
                        3,584         1  Education science medicine
                       11,744         2  Non-financial business
                        1,553         3  Non-profits/philanthropy
                        3,725         4  Other finance
                        3,038         5  PE and hedge funds
                        1,401         6  Public sector
                        1,266

In [4]:
gen nonfinance=employer_type_aggregated_num!=4 & employer_type_aggregated_num!=5
gen nonfinbiz=employer_type_aggregated_num==2
replace nonfinbiz=1 if financetype=="Insurance" | financetype=="Real estate"
gen otherfinance=employer_type_aggregated_num==4 & nonfinbiz!=1
gen pehedge=employer_type_aggregated_num==5
replace pehedge=1 if alternative==1 & year==1989
replace otherfinance=1 if pehedge!=1 & industry_jenk=="finance" & year==1989

*save d_board_members_elite_embed_long20220802, replace

(1,266 real changes made)
(28 real changes made)
(128 real changes made)


In [5]:
tab year


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       1989 |        868        3.19        3.19
       2003 |      1,567        5.76        8.95
       2004 |      1,681        6.18       15.13
       2005 |      1,673        6.15       21.28
       2006 |      1,718        6.31       27.59
       2007 |      1,742        6.40       33.99
       2008 |      1,785        6.56       40.55
       2009 |      1,777        6.53       47.08
       2010 |      1,775        6.52       53.61
       2011 |      1,802        6.62       60.23
       2012 |      1,802        6.62       66.85
       2013 |      1,800        6.62       73.47
       2014 |      1,790        6.58       80.05
       2015 |      1,839        6.76       86.81
       2016 |      1,797        6.60       93.41
       2017 |      1,793        6.59      100.00
------------+-----------------------------------
      Total |     27,209      100.00


In [6]:
gen boardtotal=1
collapse (mean) pehedge otherfinance nonfinance nonfinbiz ///
(first) schoolname schooltype (sum) boardtotal, by(unitid year)

gen schooltypenum =2 if schooltype=="public"
replace schooltypenum=3 if schooltype=="research"
label define schooltype 2 "public" 3 "private"
label values schooltypenum schooltype

(463 missing values generated)
(463 real changes made)


In [7]:
**merge endowment size data from NACUBO**
merge 1:1 unitid year using d_nacubo_funds_2003_2017, gen(b_merge) keep(1 3)
**correct university of delaware unitid to flagship
replace unitid=130943 if unitid==130343


    Result                      Number of obs
    -----------------------------------------
    Not matched                            45
        from master                        45  (b_merge==1)
        from using                          0  (b_merge==2)

    Matched                               865  (b_merge==3)
    -----------------------------------------
(15 real changes made)


In [8]:
** merge endowment data from IPEDS**
merge 1:1 unitid year using d_f1a_endow_ipeds_2003_2018, nogen keep(1 3)


    Result                      Number of obs
    -----------------------------------------
    Not matched                           494
        from master                       494  
        from using                          0  

    Matched                               416  
    -----------------------------------------


In [10]:
foreach file in board1_forbes_400_count board2_forbes_400_count /// 
mba_forbes_400_count ba_forbes_400_count jd_forbes_400_count ///
{
 qui   merge 1:1 unitid year using d_`file', gen(m`file') keep(1 3)
}

foreach var in pehedge otherfin technology inherited other forbes {
qui    egen `var'board=rowtotal(`var'board1 `var'board2)
}

foreach var in pehedge otherfin technology inherited other forbes {
qui  egen alumni`var'=rowtotal(`var'_ba `var'_mba `var'_jd)
 }

In [11]:
** merge total undergrad by race data**
**the 23 schools that don't match are 1989 private research universities from Jenkins
merge 1:1 unitid year using tef2003_2017, nogen keep(1 3) 


    Result                      Number of obs
    -----------------------------------------
    Not matched                            23
        from master                        23  
        from using                          0  

    Matched                               887  
    -----------------------------------------


In [12]:
**merge admissions rate data from IPEDS**
cd "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017"
merge 1:1 unitid year using nonmissing_sat_act_admissions_all_universities_20181223, nogen keep(1 3) keepusing(acceptrate)

/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Olig
> archy/Data/Boards 2000 to 2017

    Result                      Number of obs
    -----------------------------------------
    Not matched                            23
        from master                        23  
        from using                          0  

    Matched                               887  
    -----------------------------------------


In [13]:
*cd "/Users/Charlie/Google Drive/Lincoln Project/IPEDS Disaggregated/Data"
*merge 1:1 unitid year using hd_flags_1981_2018, keepusing(instnm control fips) keep(1 3)
*merge 1:1 unitid year using ///
* "/Users/Charlie/Dropbox/DataHub_Repos/BankersInTheIvoryTower/Chapter 5 - The Bottom - For-Profits/data/d_bankers_for_profits.dta" ///
*, keepusing(fgrnt_n fgrnt_p undertotlt underbkaat underhispt) nogen keep(1 3)

cd "/Users/Charlie/Dropbox/Stata_Notebooks/highfinancesocialcircuitry/data"
merge m:1 year using cpi_scalar2017, nogen keep(1 3)

**merge data on % students receiving federal grants (pell) from IPEDS**
merge 1:1 unitid year using finaid_2000_2017.dta, nogen keepusing(fgrnt_p) keep(1 3)

**merge fte count and first year total enrollment count data from IPEDS
merge 1:1 unitid year using fteenrollment1981_to_2018, nogen ///
keepusing(fte_count) ///
keep(1 3) // ftfirsteftotlt ptfirsteftotlt ftfirstefasiat ptfirstefasiat ftfirstefwhitt ptfirstefwhitt

**change rutgers unitid for years that it reported revenue at different level in IPEDS
replace unitid=186362 if unitid==186380 & year<2005
merge 1:1 unitid year using financial_all_1981_2017, ///
keepusing(total03_revenue) keep(1 3)

replace unitid=186380 if unitid==186362

*merge m:1 year using cpi_scalar2017, nogen keep(1 3)

/Users/Charlie/Dropbox/Stata_Notebooks/highfinancesocialcircuitry/data

    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               910  
    -----------------------------------------

    Result                      Number of obs
    -----------------------------------------
    Not matched                            23
        from master                        23  
        from using                          0  

    Matched                               887  
    -----------------------------------------

    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                               910  
    -----------------------------------------
(2 real changes made)

    Result                      Number of obs
    -----------------------------------------
    Not matched   

In [14]:
** rename NACUBO total endowment asset variable and 
** replace missing observations with IPEDS total endowment value for publics

**replace NACUBO missing endowment asset observations with IPEDS data on
rename EndowmentFunds endowment_assets
replace endowment_assets=endowment_public_ipeds if endowment_assets==.

(17 real changes made)


## list working variables before keeping only variables used in paper

In [15]:
ds, varwidth(28) alpha

Institution               inheritedboard1           pehedgeboard
State                     inheritedboard2           pehedgeboard1
_merge                    mba_forbes_400_count      pehedgeboard2
acceptrate                mboard1_forbes_400_count  schoolname
alumniforbes              mboard2_forbes_400_count  schooltype
alumniinherited           mjd_forbes_400_count      schooltypenum
alumniother               mmba_forbes_400_count     technology_ba
alumniotherfin            nonfinance                technology_jd
alumnipehedge             nonfinbiz                 technology_mba
alumnitechnology          other_ba                  technologyboard
b_merge                   other_jd                  technologyboard1
boardtotal                other_mba                 technologyboard2
cpi_scalar_2017           otherboard                tef2mort
endowment_assets          otherboard1               tefaiant
endowment_public_ipeds    otherboard2               tefasiat
fgrnt_p                

In [16]:
keep unitid year schoolname fgrnt_p acceptrate schooltypenum acceptrate ///
cpi_scalar_2017 endowment_assets total03_revenue fte_count pehedge boardtotal ///
otherfinance nonfinance nonfinbiz tefraceunder teftotlt alumnipehedge

In [17]:
label var unitid "IPEDS unique identifier"
label var year "year of observation"
label var pehedge "percent of trustees from private equity or hedge funds"
label var schoolname "name of university"
label var fgrnt_p "% undergrads receiving Pell"
label var acceptrate "undergrad admission rate"
label var schooltypenum "number indicating public (2) or private (3)"
label var cpi_scalar_2017 "current price index inflation adjustment factor to 2017 dollars"
label var endowment_assets "total endowment assets reported to NACUBO or IPEDS"
label var total03_revenue "total university revenue reported to IPEDS"
label var fte_count "total full-time equivalent student enrollment (undergrad & grad) reported to IPEDS"
label var boardtotal "total trustees"
label var pehedge "% trustees who are private equity or hedge fund managers"
label var otherfinance "% trustees with ownership or executive affiliations to other financial firms"
label var nonfinance "% trustees not associated with finance"
label var nonfinbiz "% trustees associated with non-financial businesses"
label var tefraceunder "total students Black or Latinx"
label var teftotlt "total students"
label var alumnipehedge "count of private equity and hedge fund alumni in Forbes 400"

note: label truncated to 80 characters


In [45]:
save d_school_level_elite_embed, replace

file testd_school_level_elite_embed.dta saved


In [46]:
** a pre2025 version of the dataset is saved with below name / command. 
** It should be the same but not as cleaned up.
** saving it in case cleaned up dataset breaks anything
* save pre_2025_d_school_level_elite_embed, replace

In [18]:
label values schooltypenum

describe, fullnames


Contains data
 Observations:           910                  
    Variables:            18                  
-------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
-------------------------------------------------------------------------------
unitid          double  %9.0g                 IPEDS unique identifier
year            double  %10.0g                year of observation
pehedge         float   %9.0g                 % trustees who are private equity
                                                or hedge fund managers
otherfinance    float   %9.0g                 % trustees with ownership or
                                                executive affiliations to other
                                                financial firms
nonfinance      float   %9.0g                 % trustees not associated with
                                              